In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def autoencoder(input_shape=(28, 28, 1),
                n_filters=[1, 10, 10, 10],
                filter_sizes=[3, 3, 3, 3],
                corruption=False):
    """
    Build a deep denoising autoencoder with tied weights.
    """
    # Input Layer
    input_img = tf.keras.layers.Input(shape=input_shape)
    
    # Optionally apply noise to the input images (Denoising Autoencoder)
    if corruption:
        input_img_corrupted = tf.keras.layers.GaussianNoise(stddev=0.5)(input_img)
        current_input = input_img_corrupted
    else:
        current_input = input_img

    # Encoder
    encoder_layers = []
    for n_filters, filter_size in zip(n_filters[1:], filter_sizes):
        conv = tf.keras.layers.Conv2D(n_filters, (filter_size, filter_size), activation='linear', padding='same')(current_input)
        act = tf.keras.layers.LeakyReLU()(conv)
        current_input = act
        encoder_layers.append(current_input)
    
    # Latent Representation
    z = current_input
    
    # Decoder
    for i, layer in enumerate(encoder_layers[::-1]):
        n_filters = layer.get_shape().as_list()[-1]
        if i == 0:
            upsample = tf.keras.layers.Conv2DTranspose(n_filters, (filter_sizes[-1], filter_sizes[-1]), activation='linear', padding='same')(z)
        else:
            upsample = tf.keras.layers.Conv2DTranspose(n_filters, (filter_sizes[-(i+1)], filter_sizes[-(i+1)]), activation='linear', padding='same')(current_input)
        act = tf.keras.layers.LeakyReLU()(upsample)
        current_input = act
    
    # Output Layer (Reconstruction)
    decoded = tf.keras.layers.Conv2D(input_shape[-1], (3, 3), activation='sigmoid', padding='same')(current_input)
    
    # Autoencoder Model
    autoencoder_model = tf.keras.Model(inputs=input_img, outputs=decoded)
    
    # Compile the model
    autoencoder_model.compile(optimizer='adam', loss='mse')
    
    return autoencoder_model

def test_mnist():
    """
    Test the convolutional autoencoder using MNIST dataset.
    """
    # Load MNIST dataset
    (x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
    
    # Normalize the data
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    # Reshape the data to have channels
    x_train = x_train.reshape((-1, 28, 28, 1))
    x_test = x_test.reshape((-1, 28, 28, 1))
    
    # Create the Autoencoder model
    autoencoder_model = autoencoder()
    
    # Train the model
    autoencoder_model.fit(x_train, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test, x_test))
    
    # Test the trained model
    decoded_imgs = autoencoder_model.predict(x_test)
    
    # Plot the reconstructions
    n = 10  # Number of digits to display
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # Display original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

if __name__ == '__main__':
    test_mnist()


Epoch 1/10
235/235 [==============================] - 47s 193ms/step - loss: 0.0238 - val_loss: 0.0021
Epoch 2/10
235/235 [==============================] - 53s 225ms/step - loss: 0.0014 - val_loss: 0.0010
Epoch 3/10
235/235 [==============================] - 58s 245ms/step - loss: 8.4938e-04 - val_loss: 7.5190e-04
Epoch 4/10
235/235 [==============================] - 51s 215ms/step - loss: 6.4192e-04 - val_loss: 5.5063e-04
Epoch 5/10
235/235 [==============================] - 50s 214ms/step - loss: 5.1788e-04 - val_loss: 4.6813e-04
Epoch 6/10
235/235 [==============================] - 54s 230ms/step - loss: 4.4026e-04 - val_loss: 4.3685e-04
Epoch 7/10
235/235 [==============================] - 47s 199ms/step - loss: 3.8804e-04 - val_loss: 3.6741e-04
Epoch 8/10
235/235 [==============================] - 51s 215ms/step - loss: 3.5264e-04 - val_loss: 4.0017e-04
Epoch 9/10
235/235 [==============================] - 44s 187ms/step - loss: 3.2586e-04 - val_loss: 3.0266e-04
Epoch 10/10
220/3